## Setup

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import random
import time
import warnings
from datetime import datetime

import torch

import numpy as np

import matplotlib.pyplot as plt
from scripts.differentiable_pfn_evaluation import eval_model_range
from scripts.model_builder import get_model, get_default_spec, save_model, load_model
from scripts.transformer_prediction_interface import transformer_predict, get_params_from_config, load_model_workflow

from scripts.model_configs import *

from datasets import load_openml_list, open_cc_dids, open_cc_valid_dids
from priors.utils import plot_prior, plot_features
from priors.utils import uniform_int_sampler_f

from scripts.tabular_metrics import calculate_score_per_method, calculate_score
from scripts.tabular_evaluation import evaluate

from priors.differentiable_prior import DifferentiableHyperparameterList, draw_random_style, merge_style_with_info
from scripts import tabular_metrics
from notebook_utils import *

In [3]:
large_datasets = True
max_samples = 10000 if large_datasets else 5000
bptt = 10000 if large_datasets else 3000
suite='cc'

In [4]:
device = 'cuda'
base_path = '.'
max_features = 100

In [5]:
def print_models(model_string):
    print(model_string)

    for i in range(80):
        for e in range(50):
            exists = Path(os.path.join(base_path, f'models_diff/prior_diff_real_checkpoint{model_string}_n_{i}_epoch_{e}.cpkt')).is_file()
            if exists:
                print(os.path.join(base_path, f'models_diff/prior_diff_real_checkpoint{model_string}_n_{i}_epoch_{e}.cpkt'))
        print()

In [6]:
def train_function(config_sample, i, add_name=''):
    
    torch.cuda.empty_cache()
    start_time = time.time()
    max_saved_models = 50  # 保持最多保存 50 个模型
    
    # 定义保存模型的回调函数
    def save_callback(model, epoch):
        # 仅保存第 91 到第 130 个 epoch 的模型
        if epoch < 1 or epoch > 130:
            return  # 跳过第 101 到第 150 以外的 epoch，不保存模型
        
        if not hasattr(model, 'last_saved_epoch'):
            model.last_saved_epoch = 0

        # 保存当前模型
        config_sample['epoch_in_training'] = epoch
        save_path = f'/root/autodl-fs/顺序注意力模型/prior_diff_real_checkpoint{add_name}_n_{i}_epoch_{epoch}.cpkt'
        save_model(model, base_path, save_path, config_sample)
        model.last_saved_epoch += 1  # 更新保存次数
        
        # 获取当前已保存的模型列表
        saved_models = sorted(
            [file for file in os.listdir("/root/autodl-fs/顺序注意力模型") if file.startswith(f"prior_diff_real_checkpoint{add_name}_n_{i}_")],
            key=lambda x: int(x.split('_epoch_')[-1].split('.cpkt')[0])  # 按 epoch 排序
        )
        
        # 保持最多保存 50 个模型，删除最早的模型
        if len(saved_models) > max_saved_models:
            oldest_model = saved_models[0]
            os.remove(os.path.join("/root/autodl-fs/顺序注意力模型", oldest_model))  # 删除最早的模型文件
        
    # 初始化并训练模型
    model = get_model(config_sample, device, should_train=True, verbose=1, epoch_callback=save_callback)
    
    return

## Define prior settings

In [7]:
def reload_config(config_type='causal', task_type='multiclass', longer=0, use_sequential_attention=False):
    config = get_prior_config(config_type=config_type)
    
    config['prior_type'], config['differentiable'], config['flexible'] = 'prior_bag', True, True
    
    model_string = ''
    
    config['epochs'] = 12000
    config['recompute_attn'] = True

    config['max_num_classes'] = 10
    config['num_classes'] = uniform_int_sampler_f(2, config['max_num_classes'])
    
    # 打印 num_classes 确认其值  
    print(f"Number of classes: {config['num_classes']}")  # 添加这一行 
    
    config['balanced'] = False
    model_string = model_string + '_multiclass'
    
    model_string = model_string + '_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    
    # 新增顺序注意力参数
    config['use_sequential_attention'] = use_sequential_attention
    
    return config, model_string

## Visualize Prior samples

In [8]:
config, model_string = reload_config(longer=1, use_sequential_attention=True)

config['bptt_extra_samples'] = None

# diff
config['output_multiclass_ordered_p'] = 0.
del config['differentiable_hyperparameters']['output_multiclass_ordered_p']

config['multiclass_type'] = 'rank'
del config['differentiable_hyperparameters']['multiclass_type']

config['sampling'] = 'normal' # vielleicht schlecht?
del config['differentiable_hyperparameters']['sampling']

config['pre_sample_causes'] = True
# end diff

config['multiclass_loss_type'] = 'nono' # 'compatible'
config['normalize_to_ranking'] = False # False

config['categorical_feature_p'] = .2 # diff: .0

# turn this back on in a random search!?
config['nan_prob_no_reason'] = .0
config['nan_prob_unknown_reason'] = .0 # diff: .0
config['set_value_to_nan'] = .1 # diff: 1.

config['normalize_with_sqrt'] = False

config['new_mlp_per_example'] = True
config['prior_mlp_scale_weights_sqrt'] = True
config['batch_size_per_gp_sample'] = None

config['normalize_ignore_label_too'] = False

config['differentiable_hps_as_style'] = False
config['max_eval_pos'] = 1000

config['random_feature_rotation'] = True
config['rotate_normalized_labels'] = True

config["mix_activations"] = False # False heisst eig True

config['emsize'] = 512
config['nhead'] = config['emsize'] // 128
config['bptt'] = 1024+128
config['canonical_y_encoder'] = False

    
config['aggregate_k_gradients'] = 1
config['batch_size'] = 1*config['aggregate_k_gradients']
config['num_steps'] = 1024//config['aggregate_k_gradients']
config['epochs'] = 130
config['total_available_time_in_s'] = None #60*60*22 # 22 hours for some safety...

config['train_mixed_precision'] = True
config['efficient_eval_masking'] = True

config_sample = evaluate_hypers(config)

Number of classes: <function <lambda>.<locals>.<lambda> at 0x7f3ab517b8b0>


In [9]:
#config_sample

In [10]:
#model = get_model(config_sample, device, should_train=False, verbose=2) # , state_dict=model[2].state_dict()
#(hp_embedding, data, _), targets, single_eval_pos = next(iter(model[3]))

In [11]:
#data.shape

## Training

In [ ]:
train_function(config_sample, 1, add_name='')

Using style prior: True
Using cuda device
epoch:130
steps_per_epoch:1024
batch_size:1
aggregate_k_gradients:1
梯度下降的batch_size:1
num_datasets:133120
train_mixed_precision:True
TransformerModel(
  (transformer_encoder): TransformerEncoderDiffInit(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (encoder): ModelWithAttention(
    (tabnet_encoder): TabNetModel(
      (spa